# Add a new telescope and instrument


AutoPhOT is built to be used across any telescope and their instruments. Fits files *should* follow the fits standard naming convention, although we found thoughout developent that a database is needed to translate discrepancies in naming conventions from different telescopes/instrument.

As such, AutoPhOT creates a *telescope.yml* file in the directory given in **wdir**. **wdir** house the necasscary files for automated quick photometry. To date the house the *telescope.yml* file and *catalog_queries* folder containing sequence star calibration tables

This noteboook highlights the operation of the *teledata2yml* function in the call_datacheck packge in autophot. Although this fucntion is executed during the *run* fucntion from autophot.autophot it can be executed beforehand such to take the care needed to build a reliable telescope/instrument database.

This script is used as more of a demonstartion (and the user should be aware of) rather than something the user needs to be code up themselves. This function is executed everytime AutoPhOT is executed

In [1]:
import autophot

# this is the function we will be checking out
from autophot.packages.call_datacheck import checkteledata

For the purpose of this example we will make two files from the same telescope, but from different instruments. The data will be a random image but we'll focusing on the header information.

For this demonstration we will make files from the NTT at La Silla which has EFOSC in sloan r and later in sloan g and SOFI in K band.
We will have EFOSC follow ideal header keycards whereas SOFI will follow a different paradigm.


For this demonstration we will create a folder on the users desktop and save the **fake_EFOSC2** and **fake_SOFI**. Although there is no need for the user to follow this notebook on their machine.

In [2]:
import os.path

# Get user homepath
homedir = os.path.expanduser("~")

# Get path of desktop
desktopdir = os.path.join(homedir,'Desktop')

# get path of new directory called fake fits
wdir = os.path.join(desktopdir,'fake_fits')

# create direcrory called fake_fits
os.makedirs(wdir,exist_ok = True)

Make a basic fits file with a image and header. The image will be random noise and we will all some basic keywords. First EFOSC with some standard header keywords and then SOFI where we change some keys to something more unique 

In [3]:
from numpy import random
from astropy.io import fits

# Fake EFOSC file 
data = random.random((50,50))

hdu = fits.PrimaryHDU(data)
hdul = fits.HDUList([hdu])

fake_efosc_path = os.path.join(wdir,'Fake_EFOSC2.fits')

hdul.writeto(fake_efosc_path,overwrite = True)

# update fake_efosc2.fits with usual header keywords
with fits.open(fake_efosc_path,'update') as f:
    for hdu in f:
        hdu.header['TELESCOP'] = ('ESO-NTT','added for example')
        hdu.header['INSTRUME'] = ('EFOSC2','added for example')
        hdu.header['FILTER'] = ('r','added for example')
        hdu.header['EXPTIME'] = (60.0,'added for example')
        hdu.header['GAIN'] = (1.0,'added for example')
        hdu.header['MJD-OBS'] = (58849.,'added for example')

In [4]:
# Fake SOFI file 

data = random.random((50,50))

hdu = fits.PrimaryHDU(data)
hdul = fits.HDUList([hdu])

fake_sofi_path = os.path.join(wdir,'Fake_SOFI.fits')
hdul.writeto(fake_sofi_path,overwrite = True)

# update fake_sofi.fits with some unique header keywords
with fits.open(os.path.join(wdir,'Fake_SOFI.fits'),'update') as f:
    for hdu in f:
        hdu.header['TELESCOP'] = ('ESO-NTT','added for example')
        hdu.header['INSTRUME'] = ('S_O_F_I','added for example')
        hdu.header['FIL_WHL3'] = ('K_short','added for example')
        hdu.header['EXPTIME'] = (60.0,'added for example')
        hdu.header['GAIN_E'] = (1.0,'added for example')
        hdu.header['MJD-OBS'] = (58849.,'added for example')

We can now call the checkteledata and build the telescope.yml file. checkteledata has 3 arguments:
* syntax: list of imput commands, although syntax['wdir'] is all that is needed
* flist: list of filepaths of fits files
* filepath: if telescope.yml is already create under another name fill in here

In [5]:
# create a list of filepaths  
flist = [fake_sofi_path,fake_efosc_path]
print('List of files:',flist)

List of files: ['/Users/seanbrennan/Desktop/fake_fits/Fake_SOFI.fits', '/Users/seanbrennan/Desktop/fake_fits/Fake_EFOSC2.fits']


The function relies on the syntax dictionary so we can load in the default dictionary and update our working directory (**wdir**). We can load the default file from autophot and update our wirking directory (**wdir**) from earlier.


In [6]:
# Load default synatx dictionary
from autophot.prep_input import load
syntax = load()

syntax['wdir'] = wdir

Default input loaded in from: 
/Users/seanbrennan/Documents/autophot_development/autophot_conda/autophot/autophot/databases/default_input.yml


When/if asked for corrospsonding filter name, filter name must be in this list 

'B','V','U','I','R','g','r','i','z','y','u','v','J','H','K'

e.g. rp -> r, K_short -> K
    

In [7]:
syntax = checkteledata(syntax,flist)

Header Name of Instrument [default: S_O_F_I]: 
Shorthand name [default: S_O_F_I]: SOFI
Scale units [type skip to ignore]: skip
Header Name of Instrument [default: EFOSC2]: 
Shorthand name [default: EFOSC2]: EFOSC
Scale units [type skip to ignore]: skip
Similar gain keywords found: 
['GAIN_E']
Instrument Gain key [type skip to ignore]:
[Telescope: ESO-NTT :: Inst: S_O_F_I] GAIN_E
Relevant filter keywords found:
File: Fake_SOFI.fits
*** [Key - Value] ***
FIL_WHL3 - K_short
Corrosponding FILTER Keyword
[Telescope: ESO-NTT :: Inst: INSTRUME]: FIL_WHL3
Corrosponding filter - K_short
[Telescope: ESO-NTT :: Inst: S_O_F_I][default: no_filter]: K
Similar gain keywords found: 
['GAIN']
Instrument Gain key [type skip to ignore]:
[Telescope: ESO-NTT :: Inst: EFOSC2] GAIN


Now that the telescope.yaml file has been great we can look it by loading the nest dictionary using yaml.safe_load and printing it with yaml.dump to check file structure - of course if this file is set up incorrectly you can edit it simply with a text editor

In [8]:
import yaml

# The code will look for the telescope.yml in the wdir key given in the syntax
telescope_yaml_path = os.path.join(wdir,'telescope.yml')

with open(telescope_yaml_path, 'r') as f:
   yaml_data = yaml.safe_load(f)
    
print(yaml.dump(yaml_data, allow_unicode=True, default_flow_style=False))

ESO-NTT:
  INSTRUME:
    EFOSC2:
      GAIN: GAIN
      Name: EFOSC
      filter_key_0: FILTER
      r: r
      scale_high: null
      scale_low: null
      scale_type: null
    S_O_F_I:
      GAIN: GAIN_E
      K_short: K
      Name: SOFI
      filter_key_0: FILTER
      filter_key_1: FIL_WHL3
      scale_high: null
      scale_low: null
      scale_type: null



Now that that files have been checked, if we rerun the function it won't ask up for any new entries 

In [9]:
syntax = checkteledata(syntax,flist)

If we create a new file from EFOSC2 but change the *FILTER* keyword *FIL* and the value from *r* to *gp*

In [10]:
# Fake EFOSC file 
data = random.random((50,50))

hdu = fits.PrimaryHDU(data)
hdul = fits.HDUList([hdu])

fake_efosc_g_path = os.path.join(wdir,'Fake_EFOSC2_g.fits')

hdul.writeto(fake_efosc_g_path,overwrite = True)

# update fake_efosc2.fits with usual header keywords
with fits.open(fake_efosc_g_path,'update') as f:
    for hdu in f:
        hdu.header['TELESCOP'] = ('ESO-NTT','added for example')
        hdu.header['INSTRUME'] = ('EFOSC2','added for example')
        hdu.header['FIL'] = ('gp','added for example')
        hdu.header['EXPTIME'] = (60.0,'added for example')
        hdu.header['GAIN'] = (1.0,'added for example')
        hdu.header['MJD-OBS'] = (58849.,'added for example')
        
        
        
# add this to flist
flist.append(fake_efosc_g_path)
print('Flist with new file',flist)

Flist with new file ['/Users/seanbrennan/Desktop/fake_fits/Fake_SOFI.fits', '/Users/seanbrennan/Desktop/fake_fits/Fake_EFOSC2.fits', '/Users/seanbrennan/Desktop/fake_fits/Fake_EFOSC2_g.fits']


In [11]:
syntax = checkteledata(syntax,flist)

Relevant filter keywords found:
File: Fake_EFOSC2_g.fits
*** [Key - Value] ***
FIL - gp
Corrosponding FILTER Keyword
[Telescope: ESO-NTT :: Inst: INSTRUME]: FIL
Corrosponding filter - gp
[Telescope: ESO-NTT :: Inst: EFOSC2][default: no_filter]: g


We were asked to enter the key corrosponing keyword (as any previous keyword wasn't found), filter keywords are saved as *filter_key_[1...2...3... etc]*, we were also asked to clarify what the value corrosponder to (*gp*?)

In [12]:
with open(telescope_yaml_path, 'r') as f:
   yaml_data = yaml.safe_load(f)
    
print(yaml.dump(yaml_data, allow_unicode=True, default_flow_style=False))

ESO-NTT:
  INSTRUME:
    EFOSC2:
      GAIN: GAIN
      Name: EFOSC
      filter_key_0: FILTER
      filter_key_1: FIL
      gp: g
      r: r
      scale_high: null
      scale_low: null
      scale_type: null
    S_O_F_I:
      GAIN: GAIN_E
      K_short: K
      Name: SOFI
      filter_key_0: FILTER
      filter_key_1: FIL_WHL3
      scale_high: null
      scale_low: null
      scale_type: null



Again now that all values are in telescope.yml they are no longer asked for. The script will ignore multiple files with the same details. It won't ask for multiple definitions of the same values.

In [14]:
syntax = checkteledata(syntax,flist)